In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the range of years
start_year = 2017
end_year = 2023

# Example list of department codes (replace these with actual department codes)
departments = ['1', '10', '11', '12', '13', '22', '23', '24', '25', '26', '27', '28', '29', '30', '33', '34', '36', '38', '41', '42', '46', '47', '56', '62', '63', '83', '88', 'CB', 'KU']
course_codes = set()  # Using a set to automatically handle duplicates

# Loop over each academic year
for year in range(start_year, end_year + 1):
    academic_year = f"{year}-{year+1}"
    
    # Loop over each department code
    for dept in departments:
        url = f"https://kurser.dtu.dk/archive/{academic_year}/department/{dept}"
        
        # Fetch the page content
        response = requests.get(url)
        if response.status_code == 200:  # Proceed only if the page exists
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Find all course codes (assuming they are in <td> tags inside a table row <tr>)
            rows = soup.find_all('tr')

            for row in rows:
                cells = row.find_all('td')
                if cells:
                    # Assuming the course code is in the first cell (td)
                    course_code = cells[0].get_text(strip=True)
                    course_codes.add(course_code)  # Add to the set to avoid duplicates

# Convert the set to a sorted list
course_codes_list = sorted(course_codes)

# Create a DataFrame for better visualization and export
df = pd.DataFrame(course_codes_list, columns=["Course Code"])

# Display the first few rows to check
print(df.head())

# Save to Excel or CSV for further analysis
df.to_excel('course_codes.xlsx', index=False)

  Course Code
0       01001
1       01002
2       01003
3       01004
4       01005


In [ ]:
import csv
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options

def fetch_kursusinformation_with_selenium(driver, semester, course_number):
    url = f"https://kurser.dtu.dk/course/{semester}/{course_number}"
    driver.get(url)
    
    time.sleep(2)  # Give it a moment to load fully if necessary
    
    # Get the page source and parse with BeautifulSoup
    soup = BeautifulSoup(driver.page_source, "html.parser")
    
    # Find the specific "Kursusinformation" table or section within the div
    kursusinformation_section = soup.select_one("div.box.information")
    
    if kursusinformation_section:
        # Extract all the text within the Kursusinformation section
        kursusinformation_text = kursusinformation_section.get_text(separator="\n", strip=True)
        return kursusinformation_text
    else:
        return None  # Return None if no information is found

def save_kursusinformation_to_csv(semester, course_number, kursusinformation_text, filename="kursusinformation.csv"):
    # Write Kursusinformation data to a CSV file
    with open(filename, mode='a', newline='', encoding='utf-8') as file:  # Use 'a' mode to append to the file
        writer = csv.writer(file)
        
        # Write the header only if the file is empty
        if file.tell() == 0:
            writer.writerow(["Semester", "Course Number", "Kursusinformation"])
        
        # Write the Kursusinformation with the semester and course number
        writer.writerow([semester, course_number, kursusinformation_text])

# List of semesters from 2017-2018 to 2023-2024
semesters = [f"{year}-{year+1}" for year in range(2017, 2024)]

# Generate the first 20 course numbers between 01000 to 88718
course_numbers = [str(num).zfill(5) for num in range(1000, 88718)]  # Adjust this range as needed

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--start-maximized")  # Optional: Start maximized
chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)
chrome_options.add_argument("--disable-gpu")  # Disable GPU acceleration
chrome_options.add_argument("--no-sandbox")  # Bypass OS security model

# Start the browser session once
driver = webdriver.Chrome(options=chrome_options)

try:
    # Loop through each semester and the first 20 course numbers and scrape data
    for semester in semesters:
        for course_number in course_numbers:
            kursusinformation_text = fetch_kursusinformation_with_selenium(driver, semester, course_number)
            if kursusinformation_text:
                print(f"Fetched Kursusinformation for {semester} - {course_number}")
                # Save the extracted Kursusinformation to a CSV file
                save_kursusinformation_to_csv(semester, course_number, kursusinformation_text, filename="kursusinformation.csv")
finally:
    # Close the browser session when done
    driver.quit()

print("Data fetching complete. Results saved to kursusinformation.csv")